In [90]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy

In [91]:
#This was the process used to load and clean the corpus, and produce the corrupted corpus.
#You don't need to do anything here.

corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()


def clean_word(word):
    word = word.lower().strip()
    for punctuation in ['*','"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        
        word = ''.join(word.split(punctuation))
    
    return word

corpus = [clean_word(word) for word in corpus]
corpus = [word for word in corpus if len(word) > 0]

corrupted_corpus = copy.deepcopy(corpus)

p = .25
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for k in range(len(corrupted_corpus)):
    word = corrupted_corpus[k]
    if len(word) < 2: continue
    if np.random.rand() < p:
        if np.random.randn() < 0:
            swap = np.random.choice(range(len(word)), size=2, replace = False)
            swap = np.sort(swap)
            word = ''.join([word[:swap[0]], word[swap[1]], word[swap[0]+1:swap[1]], word[swap[0]], word[swap[1]+1:]])
        else:
            
            replace = np.random.choice(range(len(word)), size=1, replace = False)[0]
            replace_with = alphabet[np.random.choice(range(len(alphabet)),size=1)[0]]
            word = ''.join([word[:replace], replace_with, word[replace+1:]])
        
        corrupted_corpus[k] = word

pickle.dump({'corpus':corpus,'corrupted_corpus':corrupted_corpus},open('alice_spelling.pkl','wb'))


In [92]:
#Take a look at how the data looks, and let's make some helper functions.
data = pickle.load(open('alice_spelling.pkl','rb'))
vocab = np.unique(data['corpus'])
V = len(vocab)


## CORRECT VS INCORRECT CORPUS
##For now, we will hold onto both the correct and incorrect corpuses. Later, you will only process the incorrect corpus, and the correct corpus is only used as a reference to check for recovery accuracy.
def recovery_rate(new_corpus, correct_corpus):
    wrong = 0
    for k in range(len(new_corpus)):
        if new_corpus[k] != correct_corpus[k]:
            wrong += 1
    return 1.- wrong/(len(new_corpus)+0.)
print('current recovery rate', recovery_rate(data['corpus'],data['corrupted_corpus'] ))

## Probability of a word mispelling
## We will use the following function to predict whether a misspelled word was actually another word. To avoid numerical issues, we make sure that the probablity is always something nonzero.
def prob_misspelled(word1,word2):
    SMALLNUM = 0.000001
    if len(word1) != len(word2): return SMALLNUM
    num_wrong = np.sum(np.array([word1[k] == word2[k] for k in range(len(word1))]))
    return np.maximum(num_wrong / (len(word1) + 0.),SMALLNUM)
print('prob misspelling alice vs alace', prob_misspelled('alice','alace'))
print('prob misspelling alice vs earth', prob_misspelled('alice','earth'))
print('prob misspelling machinelearning vs machinedreaming', prob_misspelled('machinelearning','machinedreaming'))
print('prob misspelling machinelearning vs artificalintell', prob_misspelled('machinelearning','artificalintell'))

##HASHING
#all of our objects should be vectors of length V or matrices which are V x V. 
#the kth word in the vocab list is represented by the kth element of the vector, and the relationship between the i,jth words is represented in the i,jth element in the matrix.
# to easily go between the word indices and words themselves, we need to make a hash table. 
vocab_hash = {}
for k in range(len(vocab)):
    vocab_hash[vocab[k]] = k
    
#now, to access the $k$th word, we do vocab[k]. To access the index of a word, we call vocab_hash[word].




current recovery rate 0.7683558175565884
prob misspelling alice vs alace 0.8
prob misspelling alice vs earth 1e-06
prob misspelling machinelearning vs machinedreaming 0.6666666666666666
prob misspelling machinelearning vs artificalintell 1e-06


In [93]:
## FILL ME IN ##


#WORD FREQUENCY
#create an array of length V where V[k] returns the normalized frequency of word k in the entire data corpus. Do so by filling in this function.
def get_word_freq(corpus):
    word_freq = []
    for vocab_word in vocab:
        word_freq.append(np.sum([vocab_word == word for word in corpus]))
    return np.array(word_freq)
    
    
#create an array of length V where V[k] returns the normalized frequency of word k in the entire data corpus. Do so by filling in this function.

def get_word_prob(corpus):
    word_freq = get_word_freq(corpus)
    word_prob = word_freq / (np.sum(word_freq))
    return word_prob
    
word_freq =  get_word_freq(data['corpus'])
word_prob =  get_word_prob(data['corpus'])

#report the answer of the following:
print('prob. of "alice"', word_prob[vocab_hash['alice']])
print('prob. of "queen"', word_prob[vocab_hash['queen']])
print('prob. of "chapter"', word_prob[vocab_hash['chapter']])





prob. of "alice" 0.014548615047424706
prob. of "queen" 0.002569625514869818
prob. of "chapter" 0.0009069266523069947


In [94]:


# Pr(word | prev word) 
#Using the uncorrupted corpus, accumulate the conditional transition probabilities. 
def get_transition_matrix(corpus):
    transition_matrix = np.zeros((len(vocab),len(vocab)))
    for i in range(len(corpus)-1):
        ##swap here        
        the_prev_word = corpus[i]
        word = corpus[i+1]

        word_hashed = vocab_hash[word]
        prev_word_hashed = vocab_hash[the_prev_word]

        transition_matrix[word_hashed, prev_word_hashed] += 1.

    transition_matrix = transition_matrix /  word_freq
    return transition_matrix
    
transition_matrix = get_transition_matrix(data['corpus'])


print('prob. of "the alice"', transition_matrix[vocab_hash['alice'],vocab_hash['the']])
print('prob. of "the queen"', transition_matrix[vocab_hash['queen'],vocab_hash['the']])
print('prob. of "the chapter"', transition_matrix[vocab_hash['chapter'],vocab_hash['the']])

print('prob. of "the hatter"', transition_matrix[vocab_hash['hatter'],vocab_hash['the']])



prob. of "the alice" 0.0
prob. of "the queen" 0.03970678069639585
prob. of "the chapter" 0.0
prob. of "the hatter" 0.031154551007941355


In [95]:
#The prior probabilities are just the word frequencies
prior = word_prob  + 0.

#write a function that returns the emission probability of a potentially misspelled word, by comparing its probabilities against every word in the correct vocabulary
def get_emission(mword):
    return np.array([prob_misspelled(mword,word) for word in vocab])

#find the 10 closest words to 'abice' and report them
idx = np.argsort(get_emission('abice'))[::-1]
print([vocab[j] for j in idx[:10]])


['alice', 'abide', 'voice', 'above', 'alive', 'twice', 'dunce', 'prize', 'smile', 'since']


In [96]:
#now we reduce our attention to a small segment of the corrupted corpus
corrupt_corpus =   data['corrupted_corpus'][:1000]
correct_corpus =   data['corpus'][:1000]


In [97]:
# encode the HMM spelling corrector. To debug, you can see the first hundred words of both the corrupted and proposed corpus, to see if spelling words got corrupted.

# report the recovery rate of the proposed (corrected) corpus.

#forward step
forward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)
lengthCor = len(corrupt_corpus)
i = 0

while i < lengthCor:   
    if i == 0:
        forward_probs[i,:] = prior*get_emission(corrupt_corpus[i])
        forward_probs[i,:] = forward_probs[i,:] / np.sum(forward_probs[i,:])
        i += 1
        continue

    else:
        forward_probs[i,:] = get_emission(corrupt_corpus[i])*np.dot(transition_matrix, forward_probs[i-1,:])
        forward_probs[i,:] = forward_probs[i,:] / np.sum(forward_probs[i,:])
        i += 1
        continue
    
backward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)
j = len(corrupt_corpus)-1
lengthCor2 = len(corrupt_corpus)-1

while j >= 0:
    if j == len(corrupt_corpus)-1:
        backward_probs[j,:] = get_emission(corrupt_corpus[j])
        backward_probs[j,:] = backward_probs[j,:] / np.sum(backward_probs[j,:])
        j -= 1
        continue
    else:
        backward_probs[j,:] = get_emission(corrupt_corpus[j])*np.dot(transition_matrix.T, backward_probs[j+1,:])
        backward_probs[j,:] = backward_probs[j,:] / np.sum(backward_probs[j,:])
        j -= 1
        continue

#return likelihood of kth word in corpus   
def get_max_likelihood(k):
    like = forward_probs[k,:]*backward_probs[k,:]
    return np.argmax(like)

In [98]:
#evaluate
proposed_corpus = copy.deepcopy(corrupt_corpus)
for k in range(len(corrupt_corpus)):
    new_word = vocab[get_max_likelihood(k)]
    proposed_corpus[k] = new_word

print(recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus) )
for trial in range(100):
    k = int(np.random.rand()*1000)
    if correct_corpus[k] == corrupt_corpus[k]: continue
    print(correct_corpus[k],corrupt_corpus[k],proposed_corpus[k])
    j = np.argmax(forward_probs[k,:]*backward_probs[k,:])
    i = vocab_hash[correct_corpus[k]]


0.749 0.928
deep dwep deep
she hse she
this shit this
alice aleci alice
the eht the
theyll tleylh theyll
got xot got
pop pow pop
ran rwn ran
and azd and
for fou for
was aws was
do od of
such sucs such
she seh she
she seh she
cats cqts cats
this shit this
was wsa was
or ro to
such sucs such
itself etsilf itself
this tihs this
to ot it
through nhrough through
she hse she
rabbit raibbt rabbit
alice alicx alice
the hte the
down dnwo down
